In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Loading/Cleaning VA data for expenditures in each county

In [2]:
file = pd.ExcelFile('../data/raw_data/GDX_FY21.xlsx')
states = file.sheet_names

county_exp = pd.concat([pd.read_excel(file, sheet_name=s).assign(state=s) for s in states])
county_exp.head()

,County/ Congressional District,Veteran Population*,Total Expenditure,Compensation & Pension,Construction,Education & Vocational Rehabilitation/ Employment,Loan Guaranty#,General Operating Expenses,Insurance & Indemnities,Medical Care,Unique Patients**,state
0,ALEUTIANS EAST,100.920926,91.529234,14.073,0.00000,0.000,0.0,0.00000,0.000,77.456234,*,AK
1,ALEUTIANS WEST,160.299247,493.071288,247.975,0.00000,25.985,0.0,0.00000,0.000,219.111288,26,AK
2,ANCHORAGE,28804.028026,364552.664273,142548.034,2462.27893,27128.909,0.0,23751.33971,641.747,168020.355633,9824,AK
3,BETHEL,904.079888,1708.062638,776.524,0.00000,75.095,0.0,0.00000,0.000,856.443638,76,AK
4,BRISTOL BAY,83.244817,475.606721,280.462,0.00000,22.868,0.0,0.00000,0.000,172.276721,14,AK


In [3]:
# dropping column
county_exp.drop(columns=['Compensation & Pension', 'Construction', 'Loan Guaranty#', 'General Operating Expenses', 'Insurance & Indemnities'],   inplace=True)

# cleaning column names
county_exp.columns = ['county', 'vet_pop', 'total_exp', 'edu_training_exp','medical_care_exp', 'med_patients', 'state']

# making county, state lowercase
county_exp['county'] = county_exp['county'].str.lower()
county_exp['state'] = county_exp['state'].str.lower()

#dropping duplicates
county_exp = county_exp.drop_duplicates(subset = ['state', 'county'])

county_exp.head()

,county,vet_pop,total_exp,edu_training_exp,medical_care_exp,med_patients,state
0,aleutians east,100.920926,91.529234,0.000,77.456234,*,ak
1,aleutians west,160.299247,493.071288,25.985,219.111288,26,ak
2,anchorage,28804.028026,364552.664273,27128.909,168020.355633,9824,ak
3,bethel,904.079888,1708.062638,75.095,856.443638,76,ak
4,bristol bay,83.244817,475.606721,22.868,172.276721,14,ak


In [4]:
county_exp.isna().sum()

county              0
vet_pop             0
total_exp           0
edu_training_exp    0
medical_care_exp    0
med_patients        0
state               0
dtype: int64

In [5]:
county_exp.dtypes

county               object
vet_pop             float64
total_exp           float64
edu_training_exp    float64
medical_care_exp    float64
med_patients         object
state                object
dtype: object

In [6]:
# converting column to numeric and finding missing values. 
## med_patients will not be used for initial model, so rows will not be removed at this time
county_exp['med_patients'] = county_exp['med_patients'].apply(pd.to_numeric, errors='coerce')
county_exp[county_exp['med_patients'].isna()].head()

,county,vet_pop,total_exp,edu_training_exp,medical_care_exp,med_patients,state
0,aleutians east,100.920926,91.529234,0.000,77.456234,NaN,ak
23,skagway,92.494675,330.773452,14.075,191.942452,NaN,ak
2,kalawao,20.138163,2.183321,0.000,2.183321,NaN,hi
3,kauai,4444.755221,47731.130408,1421.988,24522.226408,NaN,hi
35,greeley,63.615474,341.248764,0.000,238.246764,NaN,ks


In [7]:
county_exp.head()

,county,vet_pop,total_exp,edu_training_exp,medical_care_exp,med_patients,state
0,aleutians east,100.920926,91.529234,0.000,77.456234,NaN,ak
1,aleutians west,160.299247,493.071288,25.985,219.111288,26.0,ak
2,anchorage,28804.028026,364552.664273,27128.909,168020.355633,9824.0,ak
3,bethel,904.079888,1708.062638,75.095,856.443638,76.0,ak
4,bristol bay,83.244817,475.606721,22.868,172.276721,14.0,ak


In [8]:
# Converting vet_pop to numeric and monetary columns to true values (original data specifies *$1000)
## Do not need to be converted for modeling, so not converting now

# county_exp['vet_pop'] = county_exp['vet_pop'].astype('int')

# monetary_columns = ['total_exp', 'edu_training', 'medical_care']
# county_exp[monetary_columns]*1000

In [9]:
# adding column for total_expenditure / veteran
county_exp['exp_per_vet'] = county_exp['total_exp'] / county_exp['vet_pop']
county_exp.head()

,county,vet_pop,total_exp,edu_training_exp,medical_care_exp,med_patients,state,exp_per_vet
0,aleutians east,100.920926,91.529234,0.000,77.456234,NaN,ak,0.906940
1,aleutians west,160.299247,493.071288,25.985,219.111288,26.0,ak,3.075943
2,anchorage,28804.028026,364552.664273,27128.909,168020.355633,9824.0,ak,12.656308
3,bethel,904.079888,1708.062638,75.095,856.443638,76.0,ak,1.889283
4,bristol bay,83.244817,475.606721,22.868,172.276721,14.0,ak,5.713349


### Loading/Cleaning demographic data by county

In [10]:
education = pd.read_excel('../data/raw_data/Education - Copy.xlsx')
poverty = pd.read_excel('../data/raw_data/PovertyEstimates.xlsx')
unemployment = pd.read_excel('../data/raw_data/Unemployment.xlsx')

##### Education dataframe

In [11]:
education.head()

,State,Area name,"Percent of adults with less than a high school diploma, 2016-20","Percent of adults with a high school diploma only, 2016-20","Percent of adults completing some college or associate's degree, 2016-20",Percent of adults with a bachelor's degree or higher 2016-20
0,AL,Autauga County,11.286318,31.378763,29.017433,28.317486
1,AL,Baldwin County,9.528615,27.173557,31.390497,31.907330
2,AL,Barbour County,25.268303,35.741978,27.375401,11.614317
3,AL,Bibb County,19.115532,45.074123,24.463627,11.346719
4,AL,Blount County,17.174864,35.100719,34.472797,13.251620


In [12]:
education.isna().sum()

State                                                                        0
Area name                                                                    0
Percent of adults with less than a high school diploma, 2016-20             11
Percent of adults with a high school diploma only, 2016-20                  11
Percent of adults completing some college or associate's degree, 2016-20    11
Percent of adults with a bachelor's degree or higher 2016-20                11
dtype: int64

In [13]:
education[education['Percent of adults with less than a high school diploma, 2016-20'].isna()]

,State,Area name,"Percent of adults with less than a high school diploma, 2016-20","Percent of adults with a high school diploma only, 2016-20","Percent of adults completing some college or associate's degree, 2016-20",Percent of adults with a bachelor's degree or higher 2016-20
67,AK,Aleutian Islands,NaN,NaN,NaN,NaN
85,AK,Kuskokwim Division,NaN,NaN,NaN,NaN
93,AK,Prince of Wales-Outer Ketchikan Census Area,NaN,NaN,NaN,NaN
96,AK,Yakutat Borough,NaN,NaN,NaN,NaN
97,AK,Skagway-Hoonah-Angoon Census Area,NaN,NaN,NaN,NaN
99,AK,Upper Yukon Division,NaN,NaN,NaN,NaN
100,AK,Valdez-Cordova Census Area,NaN,NaN,NaN,NaN
102,AK,Wrangell-Petersburg Census Area,NaN,NaN,NaN,NaN
1664,MT,Yellowstone National Park,NaN,NaN,NaN,NaN
2927,VA,Bedford city,NaN,NaN,NaN,NaN


In [14]:
# cleaning column names, making strings lowercase
education.columns = ['state', 'county', 'ed_no_hs_diploma_pct', 'ed_hs_diploma_pct','ed_some_clg_pct', 'clg_grad_pct']

# making county, state lowercase
education['county'] = education['county'].str.lower()
education['state'] = education['state'].str.lower()

# condensing education columns
education['hs_grad_pct'] = 100 - education['ed_no_hs_diploma_pct']
education.drop(columns=['ed_no_hs_diploma_pct', 'ed_hs_diploma_pct', 'ed_some_clg_pct'], inplace=True)

# dropping na
education.dropna(inplace=True)

education.head()

,state,county,clg_grad_pct,hs_grad_pct
0,al,autauga county,28.317486,88.713682
1,al,baldwin county,31.907330,90.471385
2,al,barbour county,11.614317,74.731697
3,al,bibb county,11.346719,80.884468
4,al,blount county,13.251620,82.825136


##### Poverty dataframe

In [15]:
poverty.head()

,Stabr,Area_name,POVALL_2020,PCTPOVALL_2020
0,US,United States,38371394.0,11.9
1,AL,Alabama,714568.0,14.9
2,AL,Autauga County,6242.0,11.2
3,AL,Baldwin County,20189.0,8.9
4,AL,Barbour County,5548.0,25.5


In [16]:
poverty.isna().sum()

Stabr             0
Area_name         0
POVALL_2020       1
PCTPOVALL_2020    1
dtype: int64

In [17]:
poverty[poverty['POVALL_2020'].isna()]

,Stabr,Area_name,POVALL_2020,PCTPOVALL_2020
561,HI,Kalawao County,NaN,NaN


In [18]:
# dropping column
poverty.drop(columns='POVALL_2020', inplace=True)

# cleaning column names, making strings lowercase
poverty.columns = ['state', 'county', 'poverty_pct']

# making county, state lowercase
poverty['county'] = poverty['county'].str.lower()
poverty['state'] = poverty['state'].str.lower()

# dropping na
poverty.dropna(inplace=True)

poverty.head()

,state,county,poverty_pct
0,us,united states,11.9
1,al,alabama,14.9
2,al,autauga county,11.2
3,al,baldwin county,8.9
4,al,barbour county,25.5


##### Unemployment and Income dataframe

In [19]:
unemployment.head()

,State,Area_name,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Median_Household_Income_2020,Med_HH_Income_Percent_of_State_Total_2020
0,US,United States,161758337.0,153099687.0,8658650.0,5.352831,67340.0,NaN
1,AL,Alabama,2246993.0,2169721.0,77272.0,3.400000,53958.0,100.000000
2,AL,"Autauga County, AL",26341.0,25599.0,742.0,2.800000,67565.0,125.217762
3,AL,"Baldwin County, AL",99427.0,96481.0,2946.0,3.000000,71135.0,131.834019
4,AL,"Barbour County, AL",8197.0,7728.0,469.0,5.700000,38866.0,72.030097


In [20]:
unemployment.isna().sum()

State                                         0
Area_name                                     0
Civilian_labor_force_2021                     4
Employed_2021                                 4
Unemployed_2021                               4
Unemployment_rate_2021                        4
Median_Household_Income_2020                 83
Med_HH_Income_Percent_of_State_Total_2020    84
dtype: int64

In [21]:
unemployment[unemployment['Median_Household_Income_2020'].isna()].head()

,State,Area_name,Civilian_labor_force_2021,Employed_2021,Unemployed_2021,Unemployment_rate_2021,Median_Household_Income_2020,Med_HH_Income_Percent_of_State_Total_2020
94,AK,"Prince of Wales-Outer Ketchikan Census Area, AK",NaN,NaN,NaN,NaN,NaN,NaN
97,AK,"Skagway-Hoonah-Angoon Census Area, AK",NaN,NaN,NaN,NaN,NaN,NaN
99,AK,"Valdez-Cordova Census Area, AK",NaN,NaN,NaN,NaN,NaN,NaN
101,AK,"Wrangell-Petersburg Census Area, AK",NaN,NaN,NaN,NaN,NaN,NaN
3197,PR,Puerto Rico,1172111.0,1079295.0,92816.0,7.9,NaN,NaN


In [22]:
# dropping column
unemployment.drop(columns=['Civilian_labor_force_2021', 'Employed_2021', 'Unemployed_2021', 'Med_HH_Income_Percent_of_State_Total_2020'], inplace=True)

# cleaning column names, making strings lowercase
unemployment.columns = ['state', 'county', 'unemployment_pct', 'median_income']

# making county, state lowercase
unemployment['county'] = unemployment['county'].str.lower()
unemployment['state'] = unemployment['state'].str.lower()

# dropping na
unemployment.dropna(inplace=True)

# removing state abbr. from county column
unemployment['county'] = unemployment['county'].str.split(',').str[0]

unemployment.head()

,state,county,unemployment_pct,median_income
0,us,united states,5.352831,67340.0
1,al,alabama,3.400000,53958.0
2,al,autauga county,2.800000,67565.0
3,al,baldwin county,3.000000,71135.0
4,al,barbour county,5.700000,38866.0


#### Merging demographic dataframes

In [23]:
print(education.info())
print(poverty.info())
print(unemployment.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3223 entries, 0 to 3233
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   state         3223 non-null   object 
 1   county        3223 non-null   object 
 2   clg_grad_pct  3223 non-null   float64
 3   hs_grad_pct   3223 non-null   float64
dtypes: float64(2), object(2)
memory usage: 125.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3193 entries, 0 to 3193
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   state        3193 non-null   object 
 1   county       3193 non-null   object 
 2   poverty_pct  3193 non-null   float64
dtypes: float64(1), object(2)
memory usage: 99.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3193 entries, 0 to 3196
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0 

In [24]:
poverty[poverty['county'].isin(education['county']) == False].head()

,state,county,poverty_pct
0,us,united states,11.9
1,al,alabama,14.9
69,ak,alaska,9.6
116,ar,arkansas,15.2
192,ca,california,11.5


In [25]:
education[education['county'].isin(poverty['county']) == False].head()

,state,county,clg_grad_pct,hs_grad_pct
558,hi,kalawao county,39.491917,98.614319
3155,pr,puerto rico,26.671883,77.509485
3156,pr,adjuntas municipio,16.250299,65.009178
3157,pr,aguada municipio,20.536467,65.898197
3158,pr,aguadilla municipio,23.932272,73.995834


In [26]:
# merging education and poverty dataframes
ed_pov = pd.merge(education, poverty, how='inner', on=['state', 'county'])

In [27]:
ed_pov.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 0 to 3141
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   state         3142 non-null   object 
 1   county        3142 non-null   object 
 2   clg_grad_pct  3142 non-null   float64
 3   hs_grad_pct   3142 non-null   float64
 4   poverty_pct   3142 non-null   float64
dtypes: float64(3), object(2)
memory usage: 147.3+ KB


In [28]:
unemployment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3193 entries, 0 to 3196
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   state             3193 non-null   object 
 1   county            3193 non-null   object 
 2   unemployment_pct  3193 non-null   float64
 3   median_income     3193 non-null   float64
dtypes: float64(2), object(2)
memory usage: 124.7+ KB


In [29]:
ed_pov[ed_pov['county'].isin(unemployment['county']) == False].head()

,state,county,clg_grad_pct,hs_grad_pct,poverty_pct
69,ak,anchorage borough,36.579582,93.992566,8.3
79,ak,juneau borough,38.788872,96.235679,8.1
91,ak,sitka borough,34.226433,95.245875,7.4
94,ak,wrangell city and borough,16.888653,88.279169,10.6
95,ak,yakutat borough,17.990654,94.392523,12.4


In [30]:
unemployment[unemployment['county'].isin(ed_pov['county']) == False].head()

,state,county,unemployment_pct,median_income
0,us,united states,5.352831,67340.0
1,al,alabama,3.400000,53958.0
69,ak,alaska,6.400000,79961.0
72,ak,anchorage borough/municipality,5.800000,84577.0
82,ak,juneau borough/city,4.700000,89204.0


In [31]:
unemployment['county'] = unemployment['county'].str.split('/').str[0]

In [32]:
unemployment[unemployment['county'].isin(ed_pov['county']) == False].head()

,state,county,unemployment_pct,median_income
0,us,united states,5.352831,67340.0
1,al,alabama,3.400000,53958.0
69,ak,alaska,6.400000,79961.0
120,ar,arkansas,4.000000,51146.0
196,ca,california,7.300000,83001.0


In [33]:
demo = pd.merge(ed_pov, unemployment, how='inner', on=['state', 'county'])

In [34]:
demo.head()

,state,county,clg_grad_pct,hs_grad_pct,poverty_pct,unemployment_pct,median_income
0,al,autauga county,28.317486,88.713682,11.2,2.8,67565.0
1,al,baldwin county,31.907330,90.471385,8.9,3.0,71135.0
2,al,barbour county,11.614317,74.731697,25.5,5.7,38866.0
3,al,bibb county,11.346719,80.884468,17.8,3.5,50907.0
4,al,blount county,13.251620,82.825136,13.1,2.4,55203.0


In [35]:
demo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 0 to 3141
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   state             3142 non-null   object 
 1   county            3142 non-null   object 
 2   clg_grad_pct      3142 non-null   float64
 3   hs_grad_pct       3142 non-null   float64
 4   poverty_pct       3142 non-null   float64
 5   unemployment_pct  3142 non-null   float64
 6   median_income     3142 non-null   float64
dtypes: float64(5), object(2)
memory usage: 196.4+ KB


### Merge county_exp and demo dataframes

In [36]:
demo[demo['county'].isin(county_exp['county']) == False].head()

,state,county,clg_grad_pct,hs_grad_pct,poverty_pct,unemployment_pct,median_income
0,al,autauga county,28.317486,88.713682,11.2,2.8,67565.0
1,al,baldwin county,31.907330,90.471385,8.9,3.0,71135.0
2,al,barbour county,11.614317,74.731697,25.5,5.7,38866.0
3,al,bibb county,11.346719,80.884468,17.8,3.5,50907.0
4,al,blount county,13.251620,82.825136,13.1,2.4,55203.0


In [37]:
county_exp[county_exp['county'].isin(demo['county']) == False].head()

,county,vet_pop,total_exp,edu_training_exp,medical_care_exp,med_patients,state,exp_per_vet
0,aleutians east,100.920926,91.529234,0.000,77.456234,NaN,ak,0.906940
1,aleutians west,160.299247,493.071288,25.985,219.111288,26.0,ak,3.075943
2,anchorage,28804.028026,364552.664273,27128.909,168020.355633,9824.0,ak,12.656308
3,bethel,904.079888,1708.062638,75.095,856.443638,76.0,ak,1.889283
4,bristol bay,83.244817,475.606721,22.868,172.276721,14.0,ak,5.713349


In [38]:
# reformatting using delimiters
demo['county'] = demo['county'].str.split(' county').str[0]
demo['county'] = demo['county'].str.split(' borough').str[0]
demo['county'] = demo['county'].str.split(' city').str[0]
demo['county'] = demo['county'].str.split(' parish').str[0]
demo['county'] = demo['county'].str.split(' census area').str[0]
demo['county'] = demo['county'].str.split(' municipality').str[0]

#dropping duplicates
demo = demo.drop_duplicates(subset = ['state', 'county'])

In [39]:
demo[demo['county'].isin(county_exp['county']) == False]

,state,county,clg_grad_pct,hs_grad_pct,poverty_pct,unemployment_pct,median_income
72,ak,chugach,29.065300,95.646607,7.0,6.7,84063.0
73,ak,copper river,31.542553,95.425532,12.7,8.9,61215.0
97,az,arizona,30.307079,87.868093,12.8,4.9,64652.0
334,fl,desoto,12.155310,74.416109,20.8,3.8,45807.0
1417,ms,desoto,24.848520,91.010673,9.5,4.4,66532.0


In [40]:
county_exp[county_exp['county'].isin(demo['county']) == False]

,county,vet_pop,total_exp,edu_training_exp,medical_care_exp,med_patients,state,exp_per_vet
25,valdez-cordova,669.092979,6053.620653,314.383,3060.882653,191.0,ak,9.047503
24,de kalb,3421.285063,33712.051972,871.080,11560.781972,1080.0,al,9.853623
57,saint clair,6596.426381,65128.409313,1992.515,27741.810313,2074.0,al,9.873287
2,kalawao,20.138163,2.183321,0.000,2.183321,NaN,hi,0.108417
81,saint clair,25161.455096,317196.716176,24450.296,93951.849176,7083.0,il,12.606454
16,de kalb,2626.819322,26677.533806,531.400,13294.729806,886.0,in,10.155831
29,lasalle,789.447623,11088.181022,233.616,5601.030022,284.0,la,14.045493
31,de kalb,860.897088,5934.016024,175.429,3177.596024,270.0,mo,6.892829
16,carson city,4922.837848,55992.673765,1595.102,28036.701765,1937.0,nv,11.374064
20,de kalb,1128.747959,11755.196094,351.506,5545.148094,409.0,tn,10.414368


In [41]:
# merging expenditures and demo dataframes
counties = pd.merge(demo, county_exp, how='inner', on=['state', 'county'])
counties.head()

,state,county,clg_grad_pct,hs_grad_pct,poverty_pct,unemployment_pct,median_income,vet_pop,total_exp,edu_training_exp,medical_care_exp,med_patients,exp_per_vet
0,al,autauga,28.317486,88.713682,11.2,2.8,67565.0,5703.189465,75198.383682,4577.286,20057.504682,1835.0,13.185321
1,al,baldwin,31.907330,90.471385,8.9,3.0,71135.0,19665.803702,177521.631934,7375.405,50860.502934,5658.0,9.026920
2,al,barbour,11.614317,74.731697,25.5,5.7,38866.0,1848.845078,19914.886299,605.641,5575.517299,567.0,10.771528
3,al,bibb,11.346719,80.884468,17.8,3.5,50907.0,1264.859814,13536.198918,265.849,5398.731918,404.0,10.701738
4,al,blount,13.251620,82.825136,13.1,2.4,55203.0,3703.698806,31367.767836,885.531,14793.631836,1088.0,8.469309


In [42]:
counties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3121 entries, 0 to 3120
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   state             3121 non-null   object 
 1   county            3121 non-null   object 
 2   clg_grad_pct      3121 non-null   float64
 3   hs_grad_pct       3121 non-null   float64
 4   poverty_pct       3121 non-null   float64
 5   unemployment_pct  3121 non-null   float64
 6   median_income     3121 non-null   float64
 7   vet_pop           3121 non-null   float64
 8   total_exp         3121 non-null   float64
 9   edu_training_exp  3121 non-null   float64
 10  medical_care_exp  3121 non-null   float64
 11  med_patients      3107 non-null   float64
 12  exp_per_vet       3121 non-null   float64
dtypes: float64(11), object(2)
memory usage: 341.4+ KB


#### Loading and merging state VA facilities

In [43]:
fac_per_state = pd.read_csv('../data/facilities_per_state.csv')
fac_per_state.head()

,state,num_of_fac
0,AK,9
1,AL,24
2,AR,21
3,AZ,38
4,CA,102


In [44]:
fac_per_state['state'] = fac_per_state['state'].str.lower()

In [45]:
fac_per_state.rename(columns = {'num_of_fac':'state_va_fac'}, inplace=True)

In [46]:
county_df = pd.merge(counties, fac_per_state, how='left', on='state')

In [47]:
county_df.head()

,state,county,clg_grad_pct,hs_grad_pct,poverty_pct,unemployment_pct,median_income,vet_pop,total_exp,edu_training_exp,medical_care_exp,med_patients,exp_per_vet,state_va_fac
0,al,autauga,28.317486,88.713682,11.2,2.8,67565.0,5703.189465,75198.383682,4577.286,20057.504682,1835.0,13.185321,24
1,al,baldwin,31.907330,90.471385,8.9,3.0,71135.0,19665.803702,177521.631934,7375.405,50860.502934,5658.0,9.026920,24
2,al,barbour,11.614317,74.731697,25.5,5.7,38866.0,1848.845078,19914.886299,605.641,5575.517299,567.0,10.771528,24
3,al,bibb,11.346719,80.884468,17.8,3.5,50907.0,1264.859814,13536.198918,265.849,5398.731918,404.0,10.701738,24
4,al,blount,13.251620,82.825136,13.1,2.4,55203.0,3703.698806,31367.767836,885.531,14793.631836,1088.0,8.469309,24


#### Loading and cleaning county health data

In [48]:
health = pd.read_excel('../data/raw_data/2021 County Health Rankings Data - v1.xlsx', )

In [49]:
health.head()

,State,County,% Smokers,% Adults with Obesity,% Physically Inactive,% Excessive Drinking,% Severe Housing Problems,Population,% Adults with Diabetes,% Food Insecure,% Less Than 18 Years of Age,% 65 and Over,% Rural
0,Alabama,Autauga,19.810766,33.0,30.6,14.473886,13.630967,55869,12.7,15.6,23.206071,15.973080,42.002162
1,Alabama,Baldwin,18.513318,30.0,24.7,18.736958,12.996651,223234,10.3,12.9,21.302311,20.977987,42.279099
2,Alabama,Barbour,25.530768,41.2,28.0,12.466974,13.710555,24686,17.5,21.9,20.586567,19.691323,67.789635
3,Alabama,Bibb,23.084813,37.4,33.4,15.420603,9.255242,22394,13.6,15.1,20.349201,16.669644,68.352607
4,Alabama,Blount,22.954959,33.0,33.3,15.833284,10.125665,57826,14.6,13.6,22.922215,18.700930,89.951502


In [50]:
# replacing states with abbreviations
states_abbr = {
"Alabama": "AL","Alaska": "AK","Arizona": "AZ","Arkansas": "AR","California": "CA","Colorado": "CO","Connecticut": "CT","Delaware": "DE","Florida": "FL","Georgia": "GA","Hawaii": "HI",
"Idaho": "ID","Illinois": "IL","Indiana": "IN","Iowa": "IA","Kansas": "KS","Kentucky": "KY","Louisiana": "LA","Maine": "ME","Maryland": "MD","Massachusetts": "MA","Michigan": "MI",
"Minnesota": "MN","Mississippi": "MS","Missouri": "MO","Montana": "MT","Nebraska": "NE","Nevada": "NV","New Hampshire": "NH","New Jersey": "NJ","New Mexico": "NM","New York": "NY",
"North Carolina": "NC","North Dakota": "ND","Ohio": "OH","Oklahoma": "OK","Oregon": "OR","Pennsylvania": "PA","Rhode Island": "RI","South Carolina": "SC","South Dakota": "SD","Tennessee": "TN",
"Texas": "TX","Utah": "UT","Vermont": "VT","Virginia": "VA","Washington": "WA","West Virginia": "WV","Wisconsin": "WI","Wyoming": "WY","District of Columbia": "DC"}
health['State'].replace(states_abbr, inplace=True)

# cleaning column names
health.columns = ['state', 'county', 'smokers_pct', 'obesity_pct', 'inactive_pct', 'excess_alcohol_pct', 'severe_housing_problems_pct', 'county_pop', 'diabetes_pct', 'food_insecure_pct', 
                  'age_under18_pct', 'age_over65_pct', 'rural_pct']

# making county, state lowercase
health['county'] = health['county'].str.lower()
health['state'] = health['state'].str.lower()

health.head()

,state,county,smokers_pct,obesity_pct,inactive_pct,excess_alcohol_pct,severe_housing_problems_pct,county_pop,diabetes_pct,food_insecure_pct,age_under18_pct,age_over65_pct,rural_pct
0,al,autauga,19.810766,33.0,30.6,14.473886,13.630967,55869,12.7,15.6,23.206071,15.973080,42.002162
1,al,baldwin,18.513318,30.0,24.7,18.736958,12.996651,223234,10.3,12.9,21.302311,20.977987,42.279099
2,al,barbour,25.530768,41.2,28.0,12.466974,13.710555,24686,17.5,21.9,20.586567,19.691323,67.789635
3,al,bibb,23.084813,37.4,33.4,15.420603,9.255242,22394,13.6,15.1,20.349201,16.669644,68.352607
4,al,blount,22.954959,33.0,33.3,15.833284,10.125665,57826,14.6,13.6,22.922215,18.700930,89.951502


In [51]:
health.isna().sum()

state                          0
county                         0
smokers_pct                    0
obesity_pct                    0
inactive_pct                   0
excess_alcohol_pct             0
severe_housing_problems_pct    0
county_pop                     0
diabetes_pct                   0
food_insecure_pct              0
age_under18_pct                0
age_over65_pct                 0
rural_pct                      7
dtype: int64

In [52]:
health[health['rural_pct'].isna()]

,state,county,smokers_pct,obesity_pct,inactive_pct,excess_alcohol_pct,severe_housing_problems_pct,county_pop,diabetes_pct,food_insecure_pct,age_under18_pct,age_over65_pct,rural_pct
76,ak,hoonah-angoon,23.649088,30.4,16.6,21.093079,17.500000,2148,15.1,15.5,17.830540,24.022346,NaN
81,ak,kusilvak,44.571936,23.1,26.7,15.283146,69.142857,8314,4.6,28.0,41.375992,6.086120,NaN
87,ak,petersburg,19.943702,25.5,23.3,21.848153,16.334661,3266,9.2,13.8,20.238824,20.514391,NaN
88,ak,prince of wales-hyder,26.931356,40.1,19.4,19.513240,17.926566,6203,12.0,17.8,22.279542,18.136386,NaN
90,ak,skagway,16.658070,25.9,17.5,22.180733,10.588235,1183,4.7,13.0,15.384615,15.807270,NaN
93,ak,wrangell,23.142606,36.1,28.1,19.118377,13.364055,2502,18.7,13.2,19.704237,24.540368,NaN
2411,sd,oglala lakota,40.937333,40.4,26.2,18.528160,47.260274,14177,18.6,26.9,36.770826,7.427523,NaN


In [53]:
#filling na rural_pct values with mean
rural_mean = health['rural_pct'].mean()
health['rural_pct'].fillna(value=rural_mean, inplace=True)

#### Merging health data and saving the complete dataframe

In [54]:
county_df[county_df['county'].isin(health['county']) == False].head()

,state,county,clg_grad_pct,hs_grad_pct,poverty_pct,unemployment_pct,median_income,vet_pop,total_exp,edu_training_exp,medical_care_exp,med_patients,exp_per_vet,state_va_fac
2849,va,james,50.975693,95.444963,6.5,4.0,86501.0,9307.838917,117679.328197,13630.028,16169.559197,1871.0,12.643034,26
2898,va,alexandria,65.055809,93.015831,8.0,3.8,99763.0,13537.109253,100950.419022,19431.580,15648.003022,1812.0,7.457310,26
2901,va,charlottesville,55.749645,90.939652,15.8,3.7,70501.0,2142.840419,18398.358783,6278.090,3449.514783,326.0,8.585968,26
2902,va,chesapeake,34.594536,93.223568,7.6,3.8,80402.0,27857.554553,401038.356417,51580.740,70630.373417,7925.0,14.396036,26
2903,va,colonial heights,26.609583,91.824469,8.7,4.8,67339.0,1824.136414,33446.416955,1911.494,8269.112955,591.0,18.335480,26


In [55]:
health[health['county'].isin(county_df['county']) == False].head()

,state,county,smokers_pct,obesity_pct,inactive_pct,excess_alcohol_pct,severe_housing_problems_pct,county_pop,diabetes_pct,food_insecure_pct,age_under18_pct,age_over65_pct,rural_pct
92,ak,valdez-cordova,19.150134,34.0,22.2,19.590791,15.087719,9202,7.2,11.9,23.342752,14.431645,100.000000
331,fl,desoto,26.253857,31.3,35.7,20.322865,16.687952,38001,14.3,16.4,18.820557,22.520460,46.242327
547,hi,kalawao,15.509758,11.8,20.7,17.978149,7.272727,86,11.1,18.6,0.000000,45.348837,100.000000
613,il,de witt,20.605702,28.3,25.9,22.048027,8.277405,15638,18.5,9.6,21.051285,19.772349,48.753095
643,il,lasalle,21.799673,32.0,27.1,20.757786,12.778403,108669,13.2,11.0,21.285739,19.355106,30.214880


In [56]:
# reformatting county names and dropping duplicates
health['county'] = health['county'].str.split(' city').str[0]
health = health.drop_duplicates(subset=['state', 'county'])

In [57]:
county_df[county_df['county'].isin(health['county']) == False]

,state,county,clg_grad_pct,hs_grad_pct,poverty_pct,unemployment_pct,median_income,vet_pop,total_exp,edu_training_exp,medical_care_exp,med_patients,exp_per_vet,state_va_fac


In [58]:
health[health['county'].isin(county_df['county']) == False]

,state,county,smokers_pct,obesity_pct,inactive_pct,excess_alcohol_pct,severe_housing_problems_pct,county_pop,diabetes_pct,food_insecure_pct,age_under18_pct,age_over65_pct,rural_pct
92,ak,valdez-cordova,19.150134,34.0,22.2,19.590791,15.087719,9202,7.2,11.9,23.342752,14.431645,100.000000
331,fl,desoto,26.253857,31.3,35.7,20.322865,16.687952,38001,14.3,16.4,18.820557,22.520460,46.242327
547,hi,kalawao,15.509758,11.8,20.7,17.978149,7.272727,86,11.1,18.6,0.000000,45.348837,100.000000
613,il,de witt,20.605702,28.3,25.9,22.048027,8.277405,15638,18.5,9.6,21.051285,19.772349,48.753095
643,il,lasalle,21.799673,32.0,27.1,20.757786,12.778403,108669,13.2,11.0,21.285739,19.355106,30.214880
1416,ms,desoto,18.362858,37.9,26.4,15.143783,12.500000,184945,13.5,12.1,25.356187,13.249885,20.428894
2583,tx,dewitt,19.723981,25.5,32.0,18.826696,13.705234,20160,11.1,15.0,22.460317,20.014881,49.624322


In [59]:
county_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3121 entries, 0 to 3120
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   state             3121 non-null   object 
 1   county            3121 non-null   object 
 2   clg_grad_pct      3121 non-null   float64
 3   hs_grad_pct       3121 non-null   float64
 4   poverty_pct       3121 non-null   float64
 5   unemployment_pct  3121 non-null   float64
 6   median_income     3121 non-null   float64
 7   vet_pop           3121 non-null   float64
 8   total_exp         3121 non-null   float64
 9   edu_training_exp  3121 non-null   float64
 10  medical_care_exp  3121 non-null   float64
 11  med_patients      3107 non-null   float64
 12  exp_per_vet       3121 non-null   float64
 13  state_va_fac      3121 non-null   int64  
dtypes: float64(11), int64(1), object(2)
memory usage: 365.7+ KB


In [60]:
health.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3135 entries, 0 to 3140
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   state                        3135 non-null   object 
 1   county                       3135 non-null   object 
 2   smokers_pct                  3135 non-null   float64
 3   obesity_pct                  3135 non-null   float64
 4   inactive_pct                 3135 non-null   float64
 5   excess_alcohol_pct           3135 non-null   float64
 6   severe_housing_problems_pct  3135 non-null   float64
 7   county_pop                   3135 non-null   int64  
 8   diabetes_pct                 3135 non-null   float64
 9   food_insecure_pct            3135 non-null   float64
 10  age_under18_pct              3135 non-null   float64
 11  age_over65_pct               3135 non-null   float64
 12  rural_pct                    3135 non-null   float64
dtypes: float64(10), in

In [61]:
df = pd.merge(county_df, health, how='inner', on=['state', 'county'])

In [62]:
df.head()

,state,county,clg_grad_pct,hs_grad_pct,poverty_pct,unemployment_pct,median_income,vet_pop,total_exp,edu_training_exp,medical_care_exp,med_patients,exp_per_vet,state_va_fac,smokers_pct,obesity_pct,inactive_pct,excess_alcohol_pct,severe_housing_problems_pct,county_pop,diabetes_pct,food_insecure_pct,age_under18_pct,age_over65_pct,rural_pct
0,al,autauga,28.317486,88.713682,11.2,2.8,67565.0,5703.189465,75198.383682,4577.286,20057.504682,1835.0,13.185321,24,19.810766,33.0,30.6,14.473886,13.630967,55869,12.7,15.6,23.206071,15.973080,42.002162
1,al,baldwin,31.907330,90.471385,8.9,3.0,71135.0,19665.803702,177521.631934,7375.405,50860.502934,5658.0,9.026920,24,18.513318,30.0,24.7,18.736958,12.996651,223234,10.3,12.9,21.302311,20.977987,42.279099
2,al,barbour,11.614317,74.731697,25.5,5.7,38866.0,1848.845078,19914.886299,605.641,5575.517299,567.0,10.771528,24,25.530768,41.2,28.0,12.466974,13.710555,24686,17.5,21.9,20.586567,19.691323,67.789635
3,al,bibb,11.346719,80.884468,17.8,3.5,50907.0,1264.859814,13536.198918,265.849,5398.731918,404.0,10.701738,24,23.084813,37.4,33.4,15.420603,9.255242,22394,13.6,15.1,20.349201,16.669644,68.352607
4,al,blount,13.251620,82.825136,13.1,2.4,55203.0,3703.698806,31367.767836,885.531,14793.631836,1088.0,8.469309,24,22.954959,33.0,33.3,15.833284,10.125665,57826,14.6,13.6,22.922215,18.700930,89.951502


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3119 entries, 0 to 3118
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   state                        3119 non-null   object 
 1   county                       3119 non-null   object 
 2   clg_grad_pct                 3119 non-null   float64
 3   hs_grad_pct                  3119 non-null   float64
 4   poverty_pct                  3119 non-null   float64
 5   unemployment_pct             3119 non-null   float64
 6   median_income                3119 non-null   float64
 7   vet_pop                      3119 non-null   float64
 8   total_exp                    3119 non-null   float64
 9   edu_training_exp             3119 non-null   float64
 10  medical_care_exp             3119 non-null   float64
 11  med_patients                 3105 non-null   float64
 12  exp_per_vet                  3119 non-null   float64
 13  state_va_fac      

In [64]:
df = df[['county', 'state', 'total_exp', 'medical_care_exp', 'edu_training_exp', 'exp_per_vet',    #expenditures
         'county_pop', 'vet_pop', 'med_patients', 'state_va_fac',    #population info
         'hs_grad_pct', 'clg_grad_pct',      #education
         'median_income', 'poverty_pct', 'unemployment_pct', 'severe_housing_problems_pct',    #economic
         'smokers_pct', 'obesity_pct', 'inactive_pct', 'excess_alcohol_pct', 'diabetes_pct', 'food_insecure_pct',     #health
         'age_under18_pct', 'age_over65_pct', 'rural_pct'     #age, community
       ]]

In [65]:
df.to_csv('../data/county_df.csv', index=False)